In [62]:
import numpy as np
import pandas as pd

In [63]:
def node_construction(subtree,subX,suby,max_depth,min_samples_split):
    splits={}
    for index,col in subX.iteritems():
        splits[index]=np.sort(col.sample(1000,replace=True).unique())
        splits[index]=((splits[index][1:]+splits[index][:-1])/2).tolist()
    splits=[[(key,threshold) for threshold in value] for key,value in splits.items()]
    splits=[y for x in splits for y in x]

    ginis=[]
    ginis_left=[]
    ginis_right=[]
    counts_left=[]
    counts_right=[]
    for split in splits:
        mask=subX[split[0]]<split[1]
        y_left=suby[mask]
        y_right=suby[-mask]
        count_left=y_left.count()
        counts_left.append(count_left)
        count_right=y_right.count()
        counts_right.append(count_right)
        gini_left =1-((y_left.groupby( y_left).count()/ count_left)**2 ).sum()
        ginis_left.append(gini_left)
        gini_right=1-((y_right.groupby(y_right).count()/count_right)**2).sum()
        ginis_right.append(gini_right)
        gini_weighted= (y_left.count()*gini_left + y_right.count()*gini_right)/subtree['samples_count']
        ginis.append(gini_weighted)
    print(splits)
    print(ginis)

    best_split_index=ginis.index(min(ginis))
    best_split=splits[best_split_index]
    subtree['feature']=best_split[0]
    subtree['threshold']=best_split[1]
    left_gini  =ginis_left  [best_split_index]
    right_gini =ginis_right [best_split_index]
    left_count =counts_left [best_split_index]
    right_count=counts_right[best_split_index]
    mask=subX[best_split[0]]<best_split[1]
    y_left=suby[mask]
    y_right=suby[-mask]

    if left_gini < subtree['gini'] and\
      left_count >= min_samples_split and\
      subtree['level'] < max_depth and\
      left_gini > 0:
        X_left=subX[mask]
        subtree['left'] = {'level':subtree['level']+1,
                           'type':'node',
                           'gini':left_gini,
                           'samples_count':left_count,
                           'feature':None,
                           'threshold':None,
                           'left':None,
                           'right':None,
                           }
        node_construction(subtree['left'], X_left, y_left,     #recursion
                       max_depth=max_depth,
                       min_samples_split=min_samples_split)
    else:
        subtree['left'] = {'level':subtree['level']+1,
                           'type':'leaf',
                           'gini':left_gini,
                           'samples_count':left_count,
                           'category':y_left.mode()}

    if right_gini < subtree['gini'] and\
      right_count >= min_samples_split and\
      subtree['level'] < max_depth and\
      right_gini > 0:
        X_right=subX[-mask]
        subtree['right'] = {'level':subtree['level']+1,
                           'type':'node',
                           'gini':right_gini,
                           'samples_count':right_count,
                           'feature':None,
                           'threshold':None,
                           'right':None,
                           'right':None,
                           }
        node_construction(subtree['right'], X_right, y_right,   #recursion
                       max_depth=max_depth,
                       min_samples_split=min_samples_split)
    else:
        subtree['right'] = {'level':subtree['level']+1,
                           'type':'leaf',
                           'gini':right_gini,
                           'samples_count':right_count,
                           'category':y_right.mode()}
        
def node_navigation(subtree, X_sample):
    '''X_sample : df with a single line'''
    if subtree['type']=='leaf':
        return subtree['category']
    if X_sample[subtree['feature']]>subtree['threshold']:
        return node_navigation(subtree['right'], X_sample)   #recursion
    return node_navigation(subtree['left'], X_sample)        #recursion

In [73]:
class DecisionTreeClassifier():
    def __init__(self, max_depth=10000, min_samples_split=2):
        self.max_depth=max_depth
        self.min_samples_split=min_samples_split
        self.tree={'level':0,
                   'type':'root',
                   'gini':None,
                   'samples_count':None,
                   'feature':None,
                   'threshold':None,
                   'left':None,
                   'right':None,
                   }
    
    def fit(self, X_train, y_train):
        '''y_train : pd.Series. If DataFrame, the 1st column will be used as target
           X_train : DataFrame or 2D array'''
        if type(y_train)==pd.core.frame.DataFrame:
            y_train=y_train.iloc[:,0]
        X_train=pd.DataFrame(X_train)
        self.tree['gini']=1-((y_train.groupby(y_train).count()/y_train.count())**2).sum()
        self.tree['samples_count']=y_train.count()
        
        node_construction(self.tree, X_train, y_train,
                          max_depth=self.max_depth,
                          min_samples_split=self.min_samples_split)

    def predict(self, X_test):
        X_test=pd.DataFrame(X_test)
        return X_test.apply(lambda x: node_navigation(self.tree, x), axis=1)

# Fit test

In [47]:
X_train=pd.DataFrame({'aa':[0.1,0.2,0.7,0.8,0.8],
                      'bb':[1,0,1,0,0],
                      'cc':[1,0,1,1,0]})
X_train

,aa,bb,cc
0,0.1,1,1
1,0.2,0,0
2,0.7,1,1
3,0.8,0,1
4,0.8,0,0


In [48]:
y_train=pd.Series([0,0,1,1,0])
y_train

0    0
1    0
2    1
3    1
4    0
dtype: int64

In [58]:
dtc=DecisionTreeClassifier()
dtc.fit(X_train,y_train)

[('aa', 0.15000000000000002), ('aa', 0.44999999999999996), ('aa', 0.75), ('bb', 0.5), ('cc', 0.5)]
[0.4, 0.26666666666666666, 0.4666666666666666, 0.4666666666666666, 0.26666666666666666]
[('aa', 0.75), ('bb', 0.5), ('cc', 0.5)]
[0.3333333333333333, 0.3333333333333333, 0.0]


In [59]:
dtc.tree

{'level': 0,
 'type': 'root',
 'gini': 0.48,
 'samples_count': 5,
 'feature': 'aa',
 'threshold': 0.44999999999999996,
 'left': {'level': 1,
  'type': 'leaf',
  'gini': 0.0,
  'samples_count': 2,
  'category': 0    0
  dtype: int64},
 'right': {'level': 1,
  'type': 'node',
  'gini': 0.4444444444444444,
  'samples_count': 3,
  'feature': 'cc',
  'threshold': 0.5,
  'right': {'level': 2,
   'type': 'leaf',
   'gini': 0.0,
   'samples_count': 2,
   'category': 0    1
   dtype: int64},
  'left': {'level': 2,
   'type': 'leaf',
   'gini': 0.0,
   'samples_count': 1,
   'category': 0    0
   dtype: int64}}}

In [60]:
X_test=pd.DataFrame({'aa':[0.1,0.2,0.7,0.8,0.8],
                      'bb':[1,0,1,0,0],
                      'cc':[1,0,1,1,0]})

In [61]:
dtc.predict(X_test)

0    0
1    0
2    1
3    1
4    0
Name: 0, dtype: int64

# Predict test

In [81]:
dtc=DecisionTreeClassifier(min_samples_split=10)
dtc.tree={'level': 0,
 'type': 'root',
 'feature': 'aa',
 'threshold': 0.5,
 'left': None,
 'right': None}

In [82]:
dtc.tree['left']={'level': 1,
                 'type': 'root',
                 'feature': 'bb',
                 'threshold': 0.5,
                 'left': {'level':2,'type':'leaf','category':1},
                 'right': {'level':2,'type':'leaf','category':0}}
dtc.tree['right']={'level':1,
                  'type':'leaf',
                  'category':1}

In [85]:
X_test=pd.DataFrame({'aa':[0.1,0.2,0.7],'bb':[1,0,1],'cc':[1,1,0]})
X_test

,aa,bb,cc
0,0.1,1,1
1,0.2,0,1
2,0.7,1,0


In [87]:
(dtc.predict(X_test)==pd.Series([0,1,1])).all()

True

In [97]:
X_test

,aa,bb,cc
0,0.1,1,1
1,0.2,0,1
2,0.7,1,1


In [105]:
splits={}
for index,col in X_test.iteritems():
    splits[index]=np.sort(col.sample(1000,replace=True).unique())
    splits[index]=list((splits[index][1:]+splits[index][:-1])/2)
splits

{'aa': [0.15000000000000002, 0.44999999999999996], 'bb': [0.5], 'cc': []}

In [106]:
splits=[[(key,threshold) for threshold in value] for key,value in splits.items()]
splits=[y for x in splits for y in x]
splits

[('aa', 0.15000000000000002), ('aa', 0.44999999999999996), ('bb', 0.5)]

In [64]:
ser=pd.Series([4,2,3,4])
ser

0    4
1    2
2    3
3    4
dtype: int64

In [83]:
np.random.choice(ser.unique(),6)

array([2, 4, 4, 2, 2, 3])

In [89]:
np.sort(ser.sample(1000,replace=True).unique())

array([2, 3, 4])

In [77]:
npp=np.sort(ser.unique())

In [78]:
npp

array([2, 3, 4])

In [80]:
(npp[1:]+npp[:-1])/2

array([2.5, 3.5])

In [107]:
X_test

,aa,bb,cc
0,0.1,1,1
1,0.2,0,1
2,0.7,1,1


In [152]:
y_test=pd.DataFrame({'target':[1,0,1]})

In [153]:
y_test

,target
0,1
1,0
2,1


In [154]:
y_test.iloc[:,0][mask]

0    1
2    1
Name: target, dtype: int64

In [155]:
mask=X_test['bb']>0.5

In [156]:
y_left=y_test[mask].iloc[:,0]
y_left

0    1
2    1
Name: target, dtype: int64

In [175]:
1-((y_test.iloc[:,0].groupby(y_test.iloc[:,0]).count()/y_test.iloc[:,0].count())**2).sum()

0.4444444444444444

In [140]:
type(y_test)==pd.core.frame.DataFrame

True

In [117]:
mask

0    False
1    False
2     True
Name: aa, dtype: bool

In [166]:
y_test.iloc[:,0]

0    1
1    0
2    1
Name: target, dtype: int64

In [179]:
[3,4]+[3]

[3, 4, 3]

In [182]:
y_test.iloc[:,0]#.mode()

0    1
1    0
2    1
Name: target, dtype: int64

In [197]:
lis=[2,5,6,2,7]

In [198]:
lis.index(min(lis))

0